In [97]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.model_selection import train_test_split
from torch.optim import AdamW


def load_data(lines):
    dataset_dict = {}
    for line in lines:
        line = line.strip()  # \n entfernen

        words = line.split()
        if len(words) > 2:  # mindestens ein Feature + Label
            features = words[:-1]
            features = " ".join(features)
            label = words[-1].strip(".!?")  # Satzzeichen entfernen
            dataset_dict[features] = label  
        
    return dataset_dict

with open("text.txt", "r") as file:
    lines = file.readlines()



In [89]:

class TextDataset(Dataset):
    def __init__(self, data_dict, tokenizer, max_lenght= 50):

        self.data = list(data_dict.items())
        self.tokenizer = tokenizer
        self.max_length = max_lenght

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        features, label = self.data[idx]
        feature_encoding = self.tokenizer(features, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')['input_ids'].squeeze()
        label_encoding = self.tokenizer(label, truncation=True, padding='max_length', max_length=10, return_tensors='pt')['input_ids'].squeeze()
    
        return feature_encoding, label_encoding
    
def collate_fn(batch):

    feature_encoding = []
    label_encoding = []

    for feature, label in batch:
        feature_encoding.append(feature)
        label_encoding.append(label)

    features = torch.stack(feature_encoding)
    labels = torch.stack(label_encoding)

    return features, labels


In [94]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

dataset = TextDataset(load_data(lines), tokenizer)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn, shuffle=True)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TextGenerationModel(nn.Module):
    def __init__(self, emb_dim, hidden_size, output_size):
        super(TextGenerationModel, self).__init__()
        self.emb = nn.Embedding(emb_dim, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.fc3 = nn.Linear(hidden_size // 4, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.emb(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Nur die Ausgabe des letzten Zeitschritts verwenden
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    

model = TextGenerationModel(emb_dim=tokenizer.vocab_size, hidden_size=256, output_size=tokenizer.vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for i , (features, labels) in enumerate(dataloader):
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)

        target = labels[:, 0]
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

    print("Epoch finished {}/{}. Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))